# Introduction to sklearn: `fit`, `predict`, and `score`

*Scikit.learn* (also called `sklearn`) is a Python library for machine learning. The library implements a wealth of tested and highly tuned learning algorithms with a common API. 

Every algorithm thus implements a `fit` function to train the model; a `predict` function that applies the trained model on new data; and `score` to measure how well your model is doing on data for which you have gold labels.

In [5]:
%matplotlib inline
import sklearn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import codecs
import json

### Task: Professional athelete classification

To introduce the `sklearn` API, we consider a simple two-way (or binary) classification problem, the classic sumo wrestler vs baseball player:

![Sumo vs. Major League Baseball](sumo-vs-mlb.jpg)

In other words, from a pool of professional athletes, can we predict who is a sumo wrestler and who is a Major League Baseball player? The features that we'll use to make this decision are the *height* and the *weight* of the athlete.

### Data sources

The data for the task comes from two separate data sets, which we ask you to to harmonize and combine. 

#### Sumos

Data on sumo wrestlers was obtained by issuing this [query](http://tinyurl.com/m5k2ej8) on FreeBase and saved to the file `sumos.json`. It's in JSON format, an often used interchange data format that looks similar to Python syntax. 



In [6]:
sumo_json = json.load(open("sumos.json"))
sumo_json

{'result': [{'/people/person/profession': 'Sumo Wrestler',
   'date_of_birth': '1955-06-01',
   'height_meters': 1.83,
   'mid': '/m/0245_v',
   'name': 'Chiyonofuji Mitsugu',
   'type': '/people/person',
   'weight_kg': 126},
  {'/people/person/profession': 'Sumo Wrestler',
   'date_of_birth': '1969-05-08',
   'height_meters': 2.03,
   'mid': '/m/0286lq',
   'name': 'Akebono Tarō',
   'type': '/people/person',
   'weight_kg': 233},
  {'/people/person/profession': 'Sumo Wrestler',
   'date_of_birth': '1972-08-12',
   'height_meters': 1.83,
   'mid': '/m/028mbf',
   'name': 'Takanohana Kōji',
   'type': '/people/person',
   'weight_kg': 150},
  {'/people/person/profession': 'Sumo Wrestler',
   'date_of_birth': '1980-09-27',
   'height_meters': 1.84,
   'mid': '/m/028z2v',
   'name': 'Asashōryū Akinori',
   'type': '/people/person',
   'weight_kg': 154},
  {'/people/person/profession': 'Sumo Wrestler',
   'date_of_birth': '1971-05-02',
   'height_meters': 1.92,
   'mid': '/m/0291_9',
   

Below we create a pandas `DataFrame` with the sumo wrestler's dataset.

In [7]:
sumo = pd.DataFrame(sumo_json['result'])
sumo.tail()

,/people/person/profession,date_of_birth,height_meters,mid,name,type,weight_kg
95,Sumo Wrestler,1955-12-09,1.830,/m/02vn7qr,Asashio Tarō IV,/people/person,186
96,Sumo Wrestler,1958-02-15,1.780,/m/02vncr1,Tagaryū Shōji,/people/person,142
97,Sumo Wrestler,1957-04-26,1.835,/m/02vnqlq,Kotokaze Kōki,/people/person,173
98,Sumo Wrestler,1967-03-16,1.750,/m/02vq0h3,Akinoshima Katsumi,/people/person,158
99,Sumo Wrestler,1974-03-12,1.810,/m/02vrldt,Toyozakura Toshiaki,/people/person,136


#### Baseball players

The dataset with height and weight for players in Major League Baseball (MLB) was downloaded from this [HTML page](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights#SOCR_Data_-_1035_Records_of_Heights_.28in.29_and_Weights_.28lbs.29_of_Major_League_Baseball_Players), copied into a spreadsheet and exported as a csv-file.

In [8]:
mlb = pd.read_csv("mlb_heights.csv", encoding='utf-8')
mlb.tail()

,Name,Team,Position,Height_inches,Weight_pounds,Age
1029,Brad_Thompson,STL,Relief_Pitcher,73,190,25.08
1030,Tyler_Johnson,STL,Relief_Pitcher,74,180,25.73
1031,Chris_Narveson,STL,Relief_Pitcher,75,205,25.19
1032,Randy_Keisler,STL,Relief_Pitcher,75,190,31.01
1033,Josh_Kinney,STL,Relief_Pitcher,73,195,27.92


### Exercise: harmonize weight and height

The MLB and sumos datasets use different units for the weights and heights of the athletes. For the baseball player we have the height in inches and weights in pounds, whereas the registered heights are in meters and the weights in kilos.

Create (up to ) two new columns in each dataset that are consistent in both datasets:

- **weight_kg** weight in kilos
- **height_cm** height in centimeters


In [9]:
# Your code here

#### Combining MLB players and sumo wrestlers

Below we combine the the two datasets in a common `DataFrame`. 

Note that not every data point of the MLB dataset is used. Can you think of a reason for doing this?


In [10]:
sumo_vs_mlb = pd.concat([sumo[['height_cm', 'weight_kg']], 
                         mlb.ix[100:200, ['height_cm', 'weight_kg']]])
sumo_vs_mlb.tail()

KeyError: "['height_cm'] not in index"

Create a `numpy` array `is_sumo` of labels with the same length as the `sumo_vs_mlb` dataset. A value of one in this array should indicate that the corresponding row in the `sumo_vs_mlb` is a sumo wrester. In the same manner, zero means that the corresponding row is not a sumo (i.e. is an MLB player). 

In [ ]:
is_sumo = np.ones(len(sumo_vs_mlb), dtype=bool)
# Your code here

We can now visualize the distribution of the weights and heights of our dataset in 2D space

In [ ]:
fig, ax = plt.subplots()
sumo_vs_mlb.ix[is_sumo].plot(kind='scatter', x='weight_kg', y='height_cm', 
                             color='blue', label='Sumo', ax=ax)
sumo_vs_mlb.ix[~is_sumo].plot(kind='scatter', x='weight_kg', y='height_cm', 
                             color='red', label='Baseball', ax=ax);

Do you think it's possible to learn classifier that can perfectly separate the sumo and the baseball class?

### Learning a model

Step through the code below, reading the code and executing each cell. Make sure that you inspect any variable that you are curious about by **writing it in a new cell and executing that cell**. Also check the documentation on the sklearn website.

#### Converting dataset from `pandas` to `numpy`

In [ ]:
X = sumo_vs_mlb[['weight_kg', 'height_cm']].values

#### Creating a fixed train and test set

We divide the dataset into two parts. One part will be used for training, while the other part is set aside with the purpose of estimating how our model generalizes. The parts are refered to as the training and the test set. It's important not to mix these two up, e.g. using your test data to train your model. 

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, is_sumo, random_state=100)
print("Train shape", X_train.shape, "Test shape", X_test.shape)

### Fitting the classifier



In [ ]:
from sklearn.linear_model import Perceptron
perceptron = Perceptron()
perceptron

In [ ]:
perceptron.fit(X_train, y_train);

In [ ]:
perceptron.predict(X_test)

### Evaluation

In [ ]:
y_pred = perceptron.predict(X_test)
y_pred

In [ ]:
n_correct = (y_pred == y_test).sum()
print("Accuracy", n_correct / float(y_test.shape[0]))

In [ ]:
perceptron.score(X_test, y_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

print("Precision", precision_score(y_test, y_pred))
print("Recall", recall_score(y_test, y_pred))
print("F1 (balanced)", f1_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

### Estimated parameters of the model

Here we peek into the parameters of the model. This is an advanced section, so you shouldn't worry if you don't understand everything. We'll get back to this in Lecture 6. 

Do make sure that you see the decision boundary plot in the last cell.

In [ ]:
print("shape", perceptron.coef_.shape)
perceptron.coef_

In [ ]:
print("shape", perceptron.intercept_.shape)
perceptron.intercept_

#### Plotting the decision boundary

In [ ]:
def decision_boundary(w, bias, dist=0, x_start=0, x_end=300):
    y_start = -(x_start * w[0] + bias - dist) / w[1]
    y_end = -(x_end * w[0] + bias - dist) / w[1]
    return [x_start, x_end], [y_start, y_end]

In [ ]:
xx, yy = decision_boundary(perceptron.coef_[0], perceptron.intercept_[0])
fig, ax = plt.subplots()

sumo_vs_mlb[is_sumo].plot(kind='scatter', x='weight_kg', y='height_cm',
                         color='blue', label='Sumo', ax=ax)
sumo_vs_mlb[~is_sumo].plot(kind='scatter', x='weight_kg', y='height_cm',
                         color='red', label='MLB', ax=ax)
ax.set_xlim(60, 300)
ax.set_ylim(160, 210)
ax.plot(xx, yy);

### Optional exercise: Fit the data using a different classifier

Import the `LogisticRegression` classifier, using it to fit a new model. Get performance measures and plot the new decision boundary.